In [48]:
import spacy
import numpy as np
import itertools
import lemmatizer as lm
import datetime

In [3]:
%run -i trainLexicon

Negative verbs are added.
Zero infinitive forms of verbs are added.
Consonant softening forms are added.
Dropping vowel forms are added.
Becoming close vowel forms are added.
Transformed lexicon is saved to revisedDict.pkl


In [4]:
nlp_ner = spacy.load('model-best')

C:\Users\melis\anaconda3\envs\nlp\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'tr_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [830]:
doc = nlp_ner("İstanbul'dan Ankara'ya 2 Haziran'a kendim ve 2 çocuğum ve 2 yetişkin için bilet")

colors = {"DURAK": "#F67DE3", "YOLCU": "#7DF6D9", "SAYI":"#a6e22d", "AY":"#FF5733", "GÜN":"#2D14FF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [831]:
class Format:
    
    def format_weekday(self, weekday_index):
        today = datetime.date.today()
        date = today + datetime.timedelta( (weekday_index-today.weekday()) % 7 )
        date = str(date.strftime("%d-%m-%Y"))
        return date
    
    def format_delay(self, delay_type):
        today = datetime.date.today()
        date = today + datetime.timedelta(weeks=1)
        date = str(date.strftime("%d-%m-%Y"))
        return date
    
    def format_sdelay(self, delay_type, delay_count):
        today = datetime.date.today()
        #delay tagleri içerisinde geçen kelimeye göre hafta veya gün sonrasını alma
        if 'hafta' in delay_type:
            date = today + datetime.timedelta(weeks=delay_count)
        if 'gün' in delay_type:
            date = today + datetime.timedelta(days=delay_count)
        date = str(date.strftime("%d-%m-%Y"))
        return date

In [840]:
class Extract(Format):
    """
    Extract information from the received request and return a url as a response
    """
    def __init__(self):
        
        self.durak_list = ['Ankara', 'İstanbul', 'Eskişehir', 'İzmir', 'Kars', 'Konya', 'Malatya', 'Adana', 'Polatlı', 'Söğütlüçeşme', 'Eryaman', 'Bakırköy', 'Bozüyük', 'Sakarya']
        self.ay_list = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
        self.gün_list = ['pazartesi', 'salı', 'çarşamba', 'perşembe', 'cuma', 'cumartesi', 'pazar']
        self.birey_list = ['ben', 'bana', 'benim adıma', 'kendim', 'kendime']
        self.ö_erteleme_list = ['güne', 'gün sonraya', 'gün sonrasına', 'gün ilerisine', 'ay', 'hafta', 'ayın', 'haftasına']
        self.cinsiyet_list = ['erkek', 'kadın', 'kız', 'hanımefendi', 'beyefendi']
        self.erteleme_list = ['haftaya', 'günübirlik', 'haftaya yarın', 'ertesi', 'ertesi gün', 'ertesi güne', 'yarın', 'yarına', 'yarın için', 'bugün', 'bugüne', 'haftaya bugün', 'sabaha', 'akşama', 'akşam için', 'sabah için', 'öğleye', 'öğlene', 'öğleden sonraya', 'öğleden sonrasına', 'hafta içi', 'hafta sonu']
        self.yolcu_list = ['yolcu', 'engelli', 'arkadaş', 'askeri personel', 'sakat', 'gazi', 'kişilik', 'kişiye', 'şehit yakını', 'gazi yakını', 'hamile', 'çocuklu', 'arkadaşım', 'veli', 'yetişkin', 'çocuğum', 'yaşlı', 'kişi', 'çocuk', 'hayvan', 'öğrenci', 'genç', 'öğretmen', 'öğretim görevlisi', 'basın', 'muhabir', 'asker', 'personel', 'çalışan', '60 yaş', '65 yaş']
        self.sayı_list = list(np.arange(1,32).astype(str))
        
        self.response_content = {}

        self.response_content['From'] = ""
        self.response_content['To'] = ""
        
        self.response_content['DepartureDate'] = ""
        self.response_content['ReturnDate'] = ""
        
        self.response_content['PassengerType'] = []
        self.response_content['PassengerCount'] = []
        
        self.response_content['url'] = ""
        
        self.entity_list = {'DURAK':self.durak_list,
                            'AY':self.ay_list,
                            'YOLCU':self.yolcu_list,
                            'GÜN':self.gün_list,
                            'SELF':self.birey_list,
                            'DELAY':self.erteleme_list,
                            'SDELAY':self.ö_erteleme_list,
                            'CİNSİYET':self.cinsiyet_list,
                            'SAYI':self.sayı_list}
        
    def extract_entities(self, doc):
        
        duraklar = []
        aylar = []
        yolcular = []
        günler = []
        bireyler = []
        ertelemeler = []
        ö_ertelemeler = []
        ö_erteleme_sayıları = []
        cinsiyetler = []
        yolcu_sayıları = []
        tarihler = []
        
        last_ent = ""
        reverse_entities = doc.ents[::-1]
        for ent in reverse_entities:
            start = ent.start_char
            end = ent.end_char
            index = (start, end)
            label = ent.label_
            ent_list = self.entity_list[label]
            
            if label == 'DURAK' or label == 'AY' or label == 'GÜN':
                ent = lm.lemmatizeWord(str(ent))
            if label== 'YOLCU':
                ent = str(ent).split(',')[0]
                
            if str(ent) in ent_list:
                if label == 'DURAK': duraklar.append(str(ent)) 

                if label == 'AY': aylar.append(str(ent)) 

                if label == 'YOLCU': yolcular.append(str(ent)) 

                if label == 'GÜN': günler.append(str(ent)) 

                if label == 'SELF': 
                    yolcular.append(str(ent))
                    yolcu_sayıları.append(str(1))
                
                if label == 'DELAY': ertelemeler.append(str(ent)) 

                if label == 'SDELAY': ö_ertelemeler.append(str(ent)) 
                
                if label == 'CİNSİYET': cinsiyetler.append(str(ent)) 
                
                if label == 'SAYI': 
                    if last_ent == 'YOLCU':
                        yolcu_sayıları.append(str(ent))
                    elif last_ent =='SDELAY':
                        ö_erteleme_sayıları.append(str(ent))
                    else:
                        tarihler.append(str(ent))                
                    
            last_ent = label
            
        entities = [duraklar, aylar, yolcular, günler, bireyler, ertelemeler, ö_ertelemeler, ö_erteleme_sayıları, cinsiyetler, tarihler, yolcu_sayıları]
        return entities
                
    def extract_stations(self, entities):
        
        duraklar = entities[0]
        
        if len(duraklar) == 2:
            To, From = duraklar
            self.response_content['From'] = From
            self.response_content['To'] = To
            
        if len(duraklar) == 1: 
            
            # Assign default_location as the departure location that is retrieved from GPS
            From, To = 'default_location', duraklar[0]

            self.response_content['From'] = From
            self.response_content['To'] = To
            
            
    def extract_dates(self, entities):
        
        yıl = datetime.date.today().year
        tarihler = [str(tarih).zfill(2) for tarih in entities[9][::-1]]
        aylar = [str(self.ay_list.index(ay)+1).zfill(2) for ay in entities[1][::-1]]
        tam_tarih = list(itertools.zip_longest(aylar, tarihler, fillvalue=''))
        günler = [self.format_weekday(self.gün_list.index(gün)) for gün in entities[3][::-1]]
        ertelemeler = [self.format_delay(delay) for delay in entities[5][::-1]]
        #ö_erteleme_sayıları = entities[7][::-1]
        #ö_ertelemeler = self.format_sdelay(entities[6][::-1][0], int(ö_erteleme_sayıları[0]))
        
        if len(günler) == 0:
    
            if len(aylar) == 2 and len(tarihler) == 2:
                self.response_content['DepartureDate'] = f"{tam_tarih[0][1]}-{tam_tarih[0][0]}-{yıl}"
                self.response_content['ReturnDate'] = f"{tam_tarih[1][1]}-{tam_tarih[1][0]}-{yıl}"
                
            if len(aylar) == 1 and len(tarihler) == 1:
                self.response_content['DepartureDate'] = f"{tam_tarih[0][1]}-{tam_tarih[0][0]}-{yıl}" 
                
            if len(aylar) == 0 and len(tarihler) == 0:
                if len(ertelemeler) == 0 and len(ö_ertelemeler) == 0:
                    self.response_content['DepartureDate'] = 'Yarın'
                if len(ertelemeler) != 0 and len(ö_ertelemeler) == 0:
                    self.response_content['DepartureDate'] = ertelemeler[0]
                if len(ertelemeler) == 0 and len(ö_ertelemeler) != 0:
                    self.response_content['DepartureDate'] = ö_ertelemeler
         
        if len(günler) == 1:
            
            if len(aylar) == 1:
                self.response_content['DepartureDate'] = günler[0]
                self.response_content['ReturnDate'] = f"{tam_tarih[0][1]}-{tam_tarih[0][0]}-{yıl}"
                
            if len(aylar) == 2 and len(tarihler) == 2:
                self.response_content['DepartureDate'] = f"{tam_tarih[0][1]}-{tam_tarih[0][0]}-{yıl}"
                self.response_content['ReturnDate'] = f"{tam_tarih[1][1]}-{tam_tarih[1][0]}-{yıl}"
                
            elif len(aylar) == 1 and len(tarihler) == 1:
                self.response_content['DepartureDate'] = 'Yarın'
                self.response_content['ReturnDate'] = f"{tam_tarih[0][1]}-{tam_tarih[0][0]}-{yıl}"
                
            else:
                self.response_content['DepartureDate'] = günler[0]
        
            
        if len(günler) == 2:
            
            if len(aylar) == 2 and len(tarihler) == 2:
                self.response_content['DepartureDate'] = f"{tam_tarih[0][1]}-{tam_tarih[0][0]}-{yıl}"
                self.response_content['ReturnDate'] = f"{tam_tarih[1][1]}-{tam_tarih[1][0]}-{yıl}"   
                
            if len(aylar) == 0 and len(tarihler) == 0:
                self.response_content['DepartureDate'] = günler[0]
                self.response_content['ReturnDate'] = günler[1] 
    
        
    def extract_passengers(self, entities):
        
        yolcular = entities[2][::-1]
        yolcu_sayıları = entities[10][::-1]
        
        self.response_content['PassengerType'] = yolcular
        self.response_content['PassengerCount'] = yolcu_sayıları
        
    def extract(self, doc):
        entities = self.extract_entities(doc)
        self.extract_stations(entities)
        self.extract_dates(entities)
        self.extract_passengers(entities)

In [841]:
class Process(Extract):
    
    def __init__(self):
        super().__init__()
        self.passenger_vocab = {"ADT":["yolcu", "yetişkin", "kişi","kendime","kendim"],
                                "CHD":["çocuk","çocuğum"],
                                "PET":["hayvan"],
                                "YNG":["öğrenci", "genç"],
                                "TCH":["öğretmen", "öğretim görevlisi"],
                                "PRS":['basın', 'muhabir'],
                                "MLT":['asker'],
                                "STF":['personel', 'çalışan'],
                                "60Y":['60 yaş'],
                                "65+":['65 yaş']}
        self.station_vocab = {"Söğütlüçeşme":"SGTC",
                              "Ankara":"ANKR",
                              "İstanbul":"ISTN",
                              "Eskişehir":"ESKR",
                              "Polatlı":"PLTL"}
    
    def process_url(self, url_dict):
        link = "/availability?"
        #yolcuları sayılarıyla eşleştirme
        passenger_zip = list(zip(url_dict['PassengerType'], url_dict['PassengerCount']))
        for key, value in url_dict.items():
            #boş olan değerleri alma (ÖR: Dönüş tarihi verilmemişse)
            if len(value) != 0:
                if key == 'From': 
                    from_tag = self.station_vocab[value]
                    link = link+"from0"+"="+from_tag+"&"
                if key == 'To': 
                    to_tag = self.station_vocab[value]
                    link = link+"to0"+"="+to_tag+"&"
                if key == 'DepartureDate': link = link+"date0"+"="+value+"&"
                if key == 'ReturnDate': link = link+"from1"+"="+to_tag+"&"+"to1"+"="+from_tag+"&"+"date1"+"="+value+"&"
                if key == 'PassengerType': 
                    #yolcu tiplerinin ve sayılarının bulunduğu dict
                    types = {}
                    for passenger in passenger_zip:
                        for key, value in self.passenger_vocab.items(): 
                            #yolcu tipinin bulunduğu liste
                            if passenger[0] in value:
                                passenger_tag = key
                                passenger_count = int(passenger[1])
                                #eğer yolcu tagi listede varsa yolcu sayısını üstüne ekle
                                if passenger_tag in types.keys():
                                    types[passenger_tag] += passenger_count
                                #yeni bir tagse yeni yolcu ekle
                                else:
                                    types[passenger_tag] = passenger_count
                    for p, c in types.items():
                        link = link+p+"="+str(c)+"&"
        #linkteki son & işaretini alma
        link = link[:-1]
        self.response_content['url'] = link
        
    def process_dict(self, doc):
        self.extract(doc)
        self.process_url(self.response_content)

In [845]:
pr = Process()

In [846]:
pr.process_dict(doc)

In [847]:
pr.response_content

{'From': 'İstanbul',
 'To': 'Ankara',
 'DepartureDate': '02-06-2022',
 'ReturnDate': '',
 'PassengerType': ['kendim', 'çocuğum', 'yetişkin'],
 'PassengerCount': ['1', '2', '2'],
 'url': '/availability?from0=ISTN&to0=ANKR&date0=02-06-2022&ADT=3&CHD=2'}